*   Build Tkinter APP With the ability to be Complete Image Processing Control Panel with ability to
*   Button to Upload image
*   Radio Buttons for Red , Green , Blue Color Tracking or None
*   Radio Buttons for Binary ,Adaptive , Otsu  Threhsolding or None
*   Radio Buttons for Averaging , Gaussain , Median Blurring or None
*   Button for Applying Transformation

In [39]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from tkinter import *
from tkinter import filedialog
from PIL import ImageTk, Image

# The Main Functions In My Control Panel

## `select_image` funcion

In [40]:
def select_image():
    global panelA, panelB , gray, imageA
    path = filedialog.askopenfilename()

    image = cv2.imread(path)
    imageA = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # for panelA
    imageB = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # for panelB
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY) # to use it for applying any filter
    
    # Resizing the original image and edged image.
    imageA = cv2.resize(imageA, (300, 300))
    imageB = cv2.resize(imageB, (300, 300))
    gray = cv2.resize(gray, (300, 300))

    # Convert to PIL format
    imageA = Image.fromarray(imageA)
    imageB = Image.fromarray(imageB)

    # Convert to Tk format
    imageA = ImageTk.PhotoImage(imageA)
    imageB = ImageTk.PhotoImage(imageB)

    if panelA is None or panelB is None :
        panelA = Label(image = imageA)
        panelA.image = imageA
        panelA.pack(side='left', padx=10, pady=10)
        
        panelB = Label(image = imageB)
        panelB.image = imageB
        panelB.pack(side='left', padx=15, pady=10)
        
    else :
        panelA.configure(image = imageA)
        panelA.image = imageA
        
        panelB.configure(image = imageB)
        panelB.image = imageB

## `threshold` function

In [41]:
def threshold():
    thresh_type = thresholding.get()
    global gray
    #gray = cv2.cvtColor(gray, cv2.COLOR_RGB2GRAY)
    if thresh_type == 1 :
        ret, th = cv2.threshold(gray, 127, 255, cv2.THRESH_BINARY) # Binary 
        
    elif thresh_type == 2 :
        th = cv2.adaptiveThreshold(gray, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 11, 2) # adaptive

    elif thresh_type == 3 :
        blur = cv2.GaussianBlur(gray, (5,5), 0.5)
        ret, th = cv2.threshold(blur, 0, 255, cv2.THRESH_BINARY+cv2.THRESH_OTSU) #OTSU
        
    # Convert to PIL format   
    th = Image.fromarray(th)
    # Convert to Tk format
    th = ImageTk.PhotoImage(th)
    
    # Update panelB
    panelB.configure(image=th)
    panelB.image = th

## `filters` function

In [42]:
def filters():
    fltr = filtering.get()
    global gray
    
    if fltr == 1 :
        blur = cv2.blur(gray, (5,5))
    elif fltr == 2 :
        blur = cv2.GaussianBlur(gray, (5,5), 0.5)
    elif fltr == 3 :
        blur = cv2.medianBlur(gray, 5)
        
    blur = Image.fromarray(blur)
    blur = ImageTk.PhotoImage(blur)
    
    panelB.configure(image = blur)
    panelB.image = blur

## `color_tracking` function

In [43]:
    
def colors():
    color = color_tracking.get()
    cap = cv2.VideoCapture(2)
    
    while True :
        ret, frame = cap.read()
        hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    
        # define lower and upper limit of the object color choosen
        if color == 1 :
            upper_limit = np.array([0 + 15, 255, 255])
            lower_limit = np.array([0 - 15, 50, 50])
        
        elif color == 3 :
            upper_limit = np.array([120 + 15, 255, 255])
            lower_limit = np.array([120 - 15, 50, 50])
    
        elif color == 2 :
            upper_limit = np.array([60 + 15, 255, 130])
            lower_limit = np.array([60 - 15, 50, 50])
        
        # Threshold the HSV image to get only choosen color
        mask = cv2.inRange(hsv, lower_limit, upper_limit)  # Black and Wight
        res = cv2.bitwise_and(frame,frame, mask= mask)
        
        cv2.imshow('Frame', frame)
        #cv2.imshow('Mask', mask)
        cv2.imshow('Result', res)
        
        if cv2.waitKey(5) & 0XFF == 27 :
            break
        
    cap.release()
    cv2.destroyAllWindows()

## `apply` function

In [44]:
def apply():
    
    color = color_tracking.get()
    thresh_type = thresholding.get()
    fltr = filtering.get()
    
    #print(color)
    #print(thresh_type)
    
    if (color == 1) or (color == 2) or (color == 3):
        colors()
    elif color == 4 :
        pass
      
    if (thresh_type == 1) or (thresh_type == 2) or (thresh_type == 3) :
        threshold()
    elif thresh_type == 4 :
        pass
    
    if (fltr == 1) or (fltr == 2) or (fltr == 3) :
        filters()
    elif fltr == 4 :
        pass

In [47]:
# initialize the window toolkit along with the two image panels
root = Tk()
panelA = None
panelB = None
# create a button, then when pressed, will trigger a file chooser
# dialog and allow the user to select an input image; then add the
# button the GUI
btn = Button(root, text="Select an image", command=select_image) # yourapp , text ,  binded function
btn.pack(side="bottom", fill="both", expand="yes", padx="10", pady="10") #  Localization

# Radio buttons for thresholding
thresholding = IntVar()

binary = Radiobutton(root, text="Binary Thresholding", variable=thresholding, value= 1)
adaptive = Radiobutton(root, text="Adaptive Thresholding", variable=thresholding, value= 2)
otsu = Radiobutton(root, text="OTSU Thresholding", variable=thresholding, value= 3)
nothing_th = Radiobutton(root, text="None", variable=thresholding, value= 4)

binary.pack(side='right', fill='both', expand='yes', padx=3, pady=3)
adaptive.pack(side='right', fill='both', expand='yes', padx=5, pady=5)
otsu.pack(side='right', fill='both', expand='yes', padx=7, pady=7)
nothing_th.pack(side='right', fill='both', expand='yes', padx=9, pady=9)

# Radio buttons for tracking color
color_tracking = IntVar()

red = Radiobutton(root, text="Red", variable=color_tracking, value= 1)
green = Radiobutton(root, text="Green", variable=color_tracking, value= 2)
blue = Radiobutton(root, text="Blue", variable=color_tracking, value= 3)
nothing_tr = Radiobutton(root, text="None", variable=color_tracking, value= 4)

red.pack(side='bottom', fill='both', expand='yes', padx=3, pady=3)
green.pack(side='bottom', fill='both', expand='yes', padx=5, pady=5)
blue.pack(side='bottom', fill='both', expand='yes', padx=7, pady=7)
nothing_tr.pack(side='bottom', fill='both', expand='yes', padx=9, pady=9)

# Radio buttons for filtering
filtering = IntVar()

average = Radiobutton(root, text="Average Filter", variable=filtering, value= 1)
gaussian = Radiobutton(root, text="Gaussian Filter", variable=filtering, value= 2)
median = Radiobutton(root, text="Median Filter", variable=filtering, value= 3)
nothing_f = Radiobutton(root, text="None", variable=filtering, value= 4)

average.pack(side='right', fill='both', expand='yes', padx=3, pady=6)
gaussian.pack(side='right', fill='both', expand='yes', padx=5, pady=8)
median.pack(side='right', fill='both', expand='yes', padx=7, pady=10)
nothing_f.pack(side='right', fill='both', expand='yes', padx=9, pady=12)

# Apply
applybtn = Button(root, text="Apply", command=apply) # yourapp , text ,  binded function
applybtn.pack(side="bottom", fill="both", expand="yes", padx="11", pady="11") #  Localization


# kick off the GUI
root.mainloop()